In [2]:
import os
from torch.utils.data import Dataset
from torchvision import transforms
import glob
from typing import List, Dict
from utils import IOHandler
import numpy as np

In [3]:
from dataset import *

In [4]:
def pil_to_numpy(im):
    """
    Converts a PIL Image object to a NumPy array.
    Source : Fast import of Pillow images to NumPy / OpenCV arrays Written by Alex Karpinsky

    Args:
        im (PIL.Image.Image): The input PIL Image object.

    Returns:
        numpy.ndarray: The NumPy array representing the image.
    """
    im.load()

    # Unpack data
    e = Image._getencoder(im.mode, "raw", im.mode)
    e.setimage(im.im)

    # NumPy buffer for the result
    shape, typestr = Image._conv_type_shape(im)
    data = np.empty(shape, dtype=np.dtype(typestr))
    mem = data.data.cast("B", (data.data.nbytes,))

    bufsize, s, offset = 65536, 0, 0
    while not s:
        l, s, d = e.encode(bufsize)

        mem[offset : offset + len(d)] = d
        offset += len(d)
    if s < 0:
        raise RuntimeError("encoder error %d in tobytes" % s)
    return data

In [13]:
from PIL import Image

image_name = (
    "/home/ikergarcia/data/gtaai_datasets/test/car_city_day_clear/K0%3_3_3_5_5.jpeg"
)
# Read image into numpy array
image_splitter = SplitImages()
rm = RemoveMinimap(hide_map_prob=1)
image = Image.open(image_name)
image = pil_to_numpy(image)
# image = rm(image)
img = Image.fromarray(image, "RGB")
img.show()
image.shape

(270, 2400, 3)

In [11]:
image = image_splitter(image)

img = Image.fromarray(image[0], "RGB")
img.show()

In [16]:
tm = TubeMaskingGenerator((5, 270, 480), 0.9)

In [17]:
mask = tm()

In [18]:
mask.size()

torch.Size([648000])

In [13]:
from transformers import VideoMAEImageProcessor

In [14]:
image_processor = VideoMAEImageProcessor(
    do_resize=False,
    do_center_crop=False,
    do_rescale=True,
    do_normalize=True,
    image_mean=[0.33363932, 0.32581538, 0.31566033],
    image_std=[0.1914285, 0.18449214, 0.1853477],
)

In [25]:
model_inputs["pixel_values"].size()

torch.Size([1, 5, 3, 270, 480])

In [21]:
len(list(image))

5

In [1]:
from transformers import AutoImageProcessor, VideoMAEForPreTraining
import numpy as np
import torch

num_frames = 16
video = list(np.random.randint(0, 256, (num_frames, 3, 224, 224)))

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
model = VideoMAEForPreTraining.from_pretrained(
    "MCG-NJU/videomae-base", torch_dtype=torch.bfloat16
).to("cuda")

pixel_values = image_processor(video, return_tensors="pt").pixel_values

num_patches_per_frame = (model.config.image_size // model.config.patch_size) ** 2
seq_length = (num_frames // model.config.tubelet_size) * num_patches_per_frame
bool_masked_pos = torch.randint(0, 2, (1, seq_length)).bool()

outputs = model(
    pixel_values.to(device=model.device, dtype=model.dtype),
    bool_masked_pos=bool_masked_pos,
)
loss = outputs.loss

loss.backward()

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/home/ikergarcia/miniconda3/envs/transformers/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1694072516410/work/torch/csrc/utils/tensor_new.cpp:261.)
  return torch.tensor(value)


logits: torch.bfloat16
labels: torch.float32
loss: torch.float32


RuntimeError: Found dtype Float but expected BFloat16